In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

# Добавляем импорты

from gensim.models import FastText
import pandas as pd
import numpy as np
import pymorphy2
import nltk  # Natural Language Toolkit - для загрузки стоп слов русского языка
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm  # прогресбарbincount

from sklearn.model_selection import train_test_split  # рабиваем данные на traint и test

/home/avetall87/development/otus/diplom/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Создаем утилиты для работы с датасетом

In [2]:
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except Exception as e:
        return ""

In [3]:
def read_prepared_datasets(file_name) -> []:
    result = []
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file.readlines():
            # result.append(line.replace('\n', ''))
            result.append(line)
    return result

In [4]:
def process_data(data, all_stop_words) -> []:
    word_tokenizer = nltk.WordPunctTokenizer()

    texts = []
    targets = []

    # инициализируем лемматизатор
    morph = pymorphy2.MorphAnalyzer()

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):
        text_lower = words_only(item)  # оставим только слова
        tokens = word_tokenizer.tokenize(text_lower)  # разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова, а так же лемитизируем  слова
        tokens = [morph.parse(word)[0].normal_form for word in tokens if
                  (word not in all_stop_words and not word.isnumeric())]

        # texts.append(tokens) # добавляем в предобработанный список
        texts.append(' '.join(tokens))

    return texts

### Загружаем предобработанные данные

In [5]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avetall87/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
prepared_dataset_file_name = 'dataset/prepared_dataset.txt'

# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
add_stop_words = ['ао', 'оао', 'ооо']
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь',
            'декабрь', ]
all_stop_words = stop_words + add_stop_words + months

df = pd.read_csv('dataset/dataset.csv', sep=',', engine='python')

y = df['title'].tolist()

In [7]:
texts = read_prepared_datasets(prepared_dataset_file_name)
len(texts)
texts[0:3]

['подвижный сбор год днёвка деревня катеринный\n',
 'герой советский союз лётчик медноног скульптор м аникушин генеральный консул народный республика болгария ленинград вангел петрович трынок слева направо мастерская скульптор\n',
 'дважды герой соц труд смирнов избирательный участок время выборы местный совет народный судья\n']

### Загружаем модель и создаем утилиты для работы с ней

In [8]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


In [9]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [10]:
def get_text_vector(text):
    #Sentences we want sentence embeddings for
    sentences = text

    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=100, return_tensors='pt')

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    return mean_pooling(model_output, encoded_input['attention_mask'])

### Создаем векторные представления для данных

In [11]:
vectors = []
for sent in tqdm(texts):
    vectors.append(
            np.nan_to_num(np.array(get_text_vector(sent.replace('\n', ''))))[0]
        )

100%|██████████| 116000/116000 [2:57:05<00:00, 10.92it/s] 


In [12]:
df['vectors'] = vectors

In [13]:
df.head(10)

,title,text,vectors
0,6_KffdPhotoDoc_65243,Подвижной сбор 1901 года.Дневка у деревни Кате...,"[0.81877, -0.19550869, -0.6745616, 0.78428304,..."
1,6_KffdPhotoDoc_941,"Герой Советского Союза, летчик В.А.Медноногов,...","[0.46559736, -0.09520167, -0.76400936, -0.5952..."
2,6_KffdPhotoDoc_6002,Дважды Герой Соц.Труда В.А.Смирнов на избирате...,"[1.0683433, 0.024790067, -1.0115333, 0.4484961..."
3,6_KffdPhotoDoc_65096,Вид сверху на казармы полка.,"[0.21965662, -0.90563685, -0.2903497, 0.410068..."
4,6_KffdPhotoDoc_95843,Генерал казачьих войск Холмский (сидит 2-ой сл...,"[0.36982885, -0.26320565, -0.39439437, 0.20517..."
5,6_KffdPhotoDoc_66745,Секции армокаркаса у подножия бокового корпуса.,"[0.93024564, -0.250312, 0.70953155, 0.17610043..."
6,6_KffdPhotoDoc_58622,Кавалерийские части на параде войск проходят м...,"[0.35548604, -0.29775593, -0.8189927, -0.61472..."
7,6_KffdPhotoDoc_71835,Пассажирский поезд прибыл на Финляндский вокзал.,"[-0.14502977, -0.19560207, -0.6273539, -0.1823..."
8,6_KffdPhotoDoc_112688,"Хирург, профессор, орденоносец И.П.Виноградов ...","[0.19173354, -0.38422066, -0.349715, -0.097659..."
9,6_KffdPhotoDoc_18624,Скульптор В.Б. Пинчук за работой над скульптур...,"[0.5630582, -0.27837518, -1.0099745, -0.430176..."


In [14]:
np.save('dataset/vectors.npy', np.array(list(vectors), dtype=np.float64))

In [343]:
df.to_csv(path_or_buf='dataset/vectors.csv', encoding='utf-8')

In [338]:
from sklearn.metrics.pairwise import cosine_similarity

search_query = 'В каком то странном месте'.lower()
query = np.nan_to_num(np.array(get_text_vector(search_query)))

query = np.array(query)
query = np.nan_to_num(query)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(3).items():
  print({'weight':str(j), 'entity_chiper':df.title.iloc[i], 'raw text':df.text.iloc[i]})

100%|██████████| 25/25 [00:00<00:00, 3307.29it/s]


м  так    так р  н н  м  м быть  так быть
{'weight': '0.3772731423377991', 'entity_chiper': '6_KffdPhotoDoc_6890', 'raw text': 'М.И.Калинин (сидит в центре) и В.М.Молотов среди учатников конного пробега Ашхабад-Москва.'}
{'weight': '0.3751116693019867', 'entity_chiper': '6_KffdPhotoDoc_33672', 'raw text': 'Фасад дома № 116/2 - угол Знаменской улицы, 2.'}
{'weight': '0.37474656105041504', 'entity_chiper': '6_KffdPhotoDoc_40569', 'raw text': 'Кочегарка электростанции.'}


In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

request = np.nan_to_num(np.array(get_text_vector('Луна')))
print(request)

db = [
    np.nan_to_num(np.array(get_text_vector('Подвижной сбор 1901 года.Дневка у деревни Катеринен'))),
    np.nan_to_num(np.array(get_text_vector('Вид сверху на казармы полка')))
    ]
print(db)

print(db[0])
print(db[0][0])

result = []

for item in db:
    result.append(item[0])

cosine_similarities = pd.Series(cosine_similarity(request, result).flatten())
print(cosine_similarities)

NameError: name 'np' is not defined

# SQUAD

In [9]:
#Load AutoModel from huggingface model repository
from transformers import pipeline, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("./model/tokenizer_ru_bert/")
model = AutoModelForQuestionAnswering.from_pretrained("./model/pretrained_ru_bert_sbersquad/")

In [10]:
question = "Кто возлогает цветы у памятника Ленину?"
context = "Пионеры ставят корзины с цветами к подножию монумента В.И.Ленина на площади Ленина у Финляндского вокзала."

In [11]:


model = AutoModelForQuestionAnswering.from_pretrained("./model/pretrained_ru_bert_sbersquad/")

question_answerer = pipeline("question-answering", model=model,tokenizer=tokenizer)
question_answerer(question=question, context=context)

{'score': 0.9158473610877991, 'start': 0, 'end': 7, 'answer': 'Пионеры'}